# Simple Performance Analysis with TAU Commander and Hatchet

This notebook guides you through a simple performance analysis use case that measures time with [http://taucommander.paratools.com/](TAU Commander) and then manipulates the data using [https://github.com/LLNL/hatchet](Hatchet).

## Set up your environment

First, follow the Getting Started instructions in the [README](README.md).

## TAU Commander: Getting Started
The first thing we will learn is how to use TAU Commander to collect and organize performance analysis data. This guide is intended as a simple overview of the TAU Commander tool from Paratools http://taucommander.paratools.com/ for people unfamiliar with [TAU](https://www.cs.uoregon.edu/research/tau/home.php) or performance analysis in general. The authors of the TAU Commander provide much more comprehensive user guides.

In this tutorial, we will work with the P2Z benchmark, which you can obtain with:
```
git clone https://github.com/cerati/p2z-tests.git
cd p2z-tests
```
Subsequent work will take place in the p2z-tests directory tree. Make sure that the installed `taucmdr/bin` directory is in your path.

## TAU Commander's Hierarchy of Performance Measurements

TAU Commander organizes performance measurement into a set of categories intended to simplify the process of setting up performance data gathering and encourage reuse of settings between experiments.

* project - top level organization
* target - hardware environment plus measurement libraries like Papi
* application - program to test; defines what libraries and measurement techniques
* measurements
* experiments
* trials

Measurements can be configured to capture different metrics (time, hardware counters) at different granularities (functions, loops, custom regions). Typically, separate measurements are produced for each thread in a multi-threaded application and each MPI task for codes based on MPI. TAU also supports GPU measurement.

## Project

The *project* is the big umbrella that keeps everything together. It exists as a .tau directory in the directory (folder) where you create your performance experiments. Typically, TAU Commander will automatically install TAU in this `.tau` subdirectory, as well as organize the measured data in folders that store standard TAU profile files, and XML/JSON files that help TAU remeber where it put everything.

To initialize a project simply type the comand below in the directory where you wish to run your performance experiments. In your `p2z-tests` directory, create an experiments subdirectory where all subsequent steps will be taking place.

```
mkdir experiments
cd experiments
tau init # to initialize a project, this will build TAU if necessary (this can take a long time)
tau dash # to view all the pieces that currently exist within the project
```

![alt text](images/dash.png "TAU Commander Dashboard")

## Target

The *target* defines system information including OS, architecture, compilers, and external libraries, e.g., PAPI (which is needed for hardware counter data collection). Most of this information TAU can figure out on its own, but it often needs help finding a papi installation. Also, if you want to use source instrumentation add the '--pdt' flag to let TAU know.

Initially, our goal is to do simple timing, so we will create a target without any external tools. The target name can be anything, but it helps to associate it with the machine name or type. For example, on a machine called arya and using gcc compilers, we could create the target as follows:
```
tau target create arya_gcc
```

Later we can create more targets with support for PAPI counters or instrumentation (with PDT), e.g.,
```
tau target create arya_gcc_papi --papi /path/to/papi
tau target create arya_gcc_papi_pdt --papi /path/to/papi --pdt
```

## Application

Application defines application specific needs such as openmp, MPI, TBB, etc.
Note the 'T' after --tbb is for 'True'. To see all available options, run `tau application create --help`

```
tau application create p2z_tbb --linkage dynamic --tbb T
```

## Measurements

Measurements define what and how a run should record data. This includes sampling (currently recomended for this application) vs intrumentation, recording callpaths, how to store data, and most improtant what metric to measure. For TAU and our libraries to work correctly TIME must always be included and only one other metric should be measuremed at a time.

Below we have some examples of defining measurements. The --callpath 100 option is assumed by default and tells tau to record callpaths to a depth of 100.

In [ ]:
tau measurement create sample_ins_cyc --sample T --callsite T --metrics TIME PAPI_TOT_INS

## Experiments

Within a project Experiments are the primary organizational feature. Each experiment combines an application, target, and measurement so that it has all the information to run tau on your program. As a result, tau Makefiles are stored at the experiment level. 


In [ ]:
tau experiment create p2z_omp --application p2z_omp --measurement sample_ins_cyc --target arya_papi

## Trials

Program runs are called Trials, each of which is associated with the experiment that is currently selected when the Trial is created. The trials in each experiment are numbered sequentially starting at 0 and the resulting data is stored as tau profiles.

In [ ]:
tau trial create [CMD]
tau trial create ./mkFit/mkFit --cmssw-n2seeds --input-file TTbar35PU-memoryFile.fv3.clean.writeAll.recT.072617.bin --build-ce --num-thr 10 --num-events 100

## Workflow and Data Organization

 Once you decide to use Tau Performance Analysis for your program you can essentially follow this notebook from the top down.
 * Initialize the tau project
 * define the target system(s) and whether or not you wish to use pdt
 * define the application scenario (MPI, TBB, etc)
 * define all the measurements you want
 If you would like to go back later and add new measurements, systems or perhaps expand your program to include MPI this is a simple matter of creating new measurements, targets, OR applications
   * changes to the above definitions will not impact the others
   * Experiments on the other hand must be recreated if the underlying targets, measurements or applicaitons are altered

When creating experiments the goal is to consider what types of runs you will want to measure and limit experiments to varying one parameters per each. For example, we used the following experiments (and others) to explore the mkFit program.
* TTbar35 and TTbar70
  * Each runs the program once using 10 threads and 100 events from 2 different datasets
  * The Experiment is intended to get a general overview of the program as it runs on a KNL
* TTbar35_scaling
  * Similar to TTbar35 but run several times for each metric
  * Each run changes the number of threads to explore how performance is affected by additional parallelism
  * Measures "strong scaling"
* TTbar35_event_scaling
  * Similar to TTbar35_scaling but varying the number of events while holding thread count constant
  * explores how computational demand affects performance

When running Trials we care about many metrics for each, but each measurement can only support one metric at a time. To get around this issue we run the experiment several times changing the metric between each run. This process can be done using a bash script like the one shown below. A set of example scripts are available in this directory.

In [ ]:
#!/bin/bash

# parameters for mkFit
NT=10 # number of threads
NE=20 # number of simulated events
FILE=TTbar70PU-memoryFile.fv3.clean.writeAll.recT.072617.bin # input file name

# parameters for TAU
EXP=demo  # experiment name - must be created before running this script
tau experiment select $EXP # switch tau to this experiment

# command to execute the mkFit program
#    note the '\' before each space
#    also this must be redefined inside the loop to overwrite the variable as they vary
CMD=./mkFit/mkFit\ --cmssw-n2seeds\ --input-file\ $FILE\ --build-ce\ --num-thr\ $NT\ --num-events\ $NE

measure_list=(scalar_simd packed_simd tot_cyc tot_ins tcm2 res_stl tca2 lst tcm1 llc_ref llc_miss)


function run_trials {

    for i in ${measure_list[@]}; do
      tau experiment edit $EXP --measurement $i
      tau trial create $CMD
    done

}

run_trials

